In [1]:
import praw
from praw.exceptions import InvalidURL
from praw.models import Submission, Comment

from data_user_comments import get_context, format_comment_as_json, format_submission_as_json
from finetune import get_qa_string
from infertools import get_text_generation_pipeline

In [2]:
def make_package_comment(comment):
    _, submission = get_context(comment, reddit)
    return {
        'comment': None,
        'parent_comment': format_comment_as_json(comment),
        'submission': format_submission_as_json(submission)
    }

def make_package_submission(submission):
    return {
        'comment': None,
        'parent_comment': None,
        'submission': format_submission_as_json(submission)
    }

def make_package_url(url, reddit):
    try:
        refobj = reddit.comment(url=url)
    except InvalidURL:
        refobj = reddit.submission(url=url)

    if isinstance(refobj, Submission):
        return make_package_submission(refobj)
    elif isinstance(refobj, Comment):
        return make_package_comment(refobj)

In [3]:
reddit = praw.Reddit()

In [4]:
#url = 'https://www.reddit.com/r/Fedora/comments/g62avm/fedora_32_nvidia_drivers_install/' # submission
url = 'https://www.reddit.com/r/apple/comments/ae5cm7/apple_hires_prominent_facebook_critic_for/edn5hmg/' # comment

In [5]:
package = make_package_url(url, reddit)
prompt = get_qa_string(package)

In [6]:
txtgen = get_text_generation_pipeline('finetune/spez/model')

In [7]:
# https://huggingface.co/transformers/_modules/transformers/pipelines.html#TextGenerationPipeline
# https://huggingface.co/transformers/_modules/transformers/modeling_utils.html#PreTrainedModel.generate
responses = txtgen(prompt, num_return_sequences=1, max_length=1024)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


In [8]:
print (prompt)

In subreddit: apple
Title: Apple Hires Prominent Facebook Critic for Internal-Facing Product Privacy Role


Q: If that's what you want then too bad. Unless another company can do this you're stuck with Apple and paying Apple prices.
A: 


In [10]:
print (responses[0]['generated_text'].replace(prompt, '').strip().split('\n')[0])

A CEO and a CEO, all with a mission to improve the world and everyone standing up for their communities, are perfectly fine with this. But please, do not take me as someone who supports or outright opposes the company. I'm a little kid running through a long day, and it just made me riled up to see how the site looked when he took it off.
